# Resource Catalog

## Remove in production release
This notebook demostrates usage of the EODH resource catalog API using `pyeodh`

### Ensure 
- a clear description of purpose, 
- intended audience and/or use case 
- linkages between notebooks and other training resources (if required) 

### Record 
- Technical dependencies, 
- Platform and Service Dependencies, 
- Python Language versions, 
- libraries, additional scripts and files.

### Remember
"When Jupyter notebooks are used in an educational context, they should not only be conceptualized to teach a specific topic but should also set a good example by following and implementing best practices for scientific computing" 

- Need in-order execution of notebook cells 
- Good-quality code 
- No code duplication 
- Imports at the beginning of a notebook
- Consistent code style and formatting
- Meaningful names for variables
- Licence for code and training resources

---

__Description & purpose__:

__Author(s)__:

__Date created__:

__Date last modified__:

__Licence__: This notebook is released under Creative Commons .... The code is released using ...


First we need to create an instance of the `Client`, which is our entrypoint to EODH APIs.

In [1]:
import pyeodh

client = pyeodh.Client(base_url="https://test.eodatahub.org.uk") # Optionally specify a different server

Fetch the resource catalog object.


In [2]:
# GET /stac-fastapi/catalogs
service = client.get_catalog_service()
catalogs = service.get_catalogs()
ceda_cat = service.get_catalog("supported-datasets/ceda-stac-fastapi")

HTTPError: 500 Server Error: Internal Server Error for url: https://test.eodatahub.org.uk/api/catalogue/stac/catalogs

All attributes are mapped to properties, e.g.

In [13]:
print("id: ", catalogs.id)
#print("title: ", rc.title)
#print("description: ", rc.description)

AttributeError: 'list' object has no attribute 'id'

API endpoints are wrapped in methods and are structured into classes following the same logic as the API. E.g. to fetch a collection item, I first need to get the collection from the resource catalog.

In [ ]:
# GET /stac-fastapi/collections/{collection_id}/items/{item_id}
cmip6 = rc.get_collection("cmip6")
item = cmip6.get_item("CMIP6.ScenarioMIP.THU.CIESM.ssp585.r1i1p1f1.Amon.rsus.gr.v20200806")
print(item.id)

Some API responses are paginated (e.g. collection items), and you can simply iterate over them.

In [ ]:
# GET /stac-fastapi/collections/cmip6/items
items = cmip6.get_items()
for item in items:
    print(item.id)

Attempting to create a collection with id that already exists will result in 409 error code. To see the example in action delete the test collection first by running the cell below.

Delete a colletion

In [ ]:
rc.get_collection("test1").delete()

Create new collection example

In [ ]:
test1 = rc.create_collection(id="test1", title="Test", description="Test collection")
print(test1.description)

Update a collection

In [ ]:
test1.update(description="Different description")
print(test1.description)

Create an item

In [ ]:
testitem1 = test1.create_item(id="test1.testitem1")
print(f"Created {testitem1.id} in collection {testitem1.collection}")

Update an item

In [ ]:
testitem1.update(properties={"foo": "bar"})
print(testitem1.properties)

Delete an item

In [ ]:
testitem1.delete()

Find out more about the Resource Catalog

In [ ]:
print(f"Livecheck: PING-{rc.ping()}")
print("\nAPI conforms to:", *rc.get_conformance(), sep="\n")

Search the Catalog

In [ ]:
for result in rc.search(collections=['cmip6']):
    print(result.id)